In [33]:
import numpy as np
import scipy as sp
import math
# Define known parameters
rho1 = 1553.6; mu1 = 65e-3; N1 = 14000; D1 = 50e-3; M1 = 0.85; P1 = 10.4e5
#rho2 = 1000; mu2 = 10e-3
rho2 = 1286; mu2 = 33.82e-3 #60% Sucrose W/W at 30 C
g = 9.807  # Acceleration due to gravity in m/s^2

Q1 = M1/rho1

# Define lower and upper bounds for D, N, m_dot, pressure rise
lb = [50e-3, 4000, 0.1, 1e5]  # Lower bounds
ub = [55e-3, 8000, 1.0, 3e5]  # Upper bounds
bounds = [(low, high) for low, high in zip(lb, ub)]
#np.random.seed(100)

# Define the objective function (sum of squared residuals)
def pump_equations(x):
    D2, N2, M2, P2 = x
    
    # Specific speed equation residual
    F1 = (N1 * np.sqrt(M1/rho1) / (P1/(rho1 * g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (P2/(rho2 * g))**(3/4))
    
    # Flow coefficient equation residual
    F2 = ( M1/rho1 / (N1 * D1**3)) - (M2/rho2 / (N2 * D2**3))
    
    # Head coefficient equation residual
    F3 = ( P1/rho1 / (N1**2 * D1**2)) - ( P2/rho2 / (N2**2 * D2**2))
    
    # Reynolds number equation residual
    F4 = (M1/mu1 / D1) - (M2/mu2 / D2)
    
    # Power Coefficient (based on hydraulic power)
    F5 = ( P1 * M1/rho1 / ( rho1 * N1**3 * D1**5) ) - ( P2 * M2/rho2 / ( rho2 * N2**3 * D2**5) )
    
    # Sum of squared residuals
    #return F1**2 + F2**2 + F3**2 + F4**2
    return [F1, F2, F3, F4, F5]

# Normalization function for better scaling
def normalize_residuals(residuals):
    # Scale each residual by a factor that brings them into a similar range
    scales = [1.0, 1.0e-3, 1.0e-3, 1, 1.0e3]  # Example scales, adjust as needed
    return [residuals[i] / scales[i] for i in range(len(residuals))]

# Objective function for minimization (sum of squares of normalized residuals)
def objective_function(x):
    residuals = pump_equations(x)
    normalized_residuals = normalize_residuals(residuals)
    return sum([r**2 for r in normalized_residuals])


# Initial guesses for unknowns
x0 = np.array([50e-3, 7000, 0.3, 2e5])

# Perform optimization using 'SLSQP' (Sequential Least Squares Programming)
#result = sp.optimize.minimize(objective_function, x0, bounds=bounds, method='trust-constr', options={'disp': True, 'maxiter': 10000, 'xtol': 1e-16, 'gtol': 1e-16})
#result = sp.optimize.differential_evolution(objective_function, bounds, disp = True, popsize=300, tol = 1e-16, maxiter = 1000)
#result = sp.optimize.root(pump_equations, x0, method='trust-constr', bounds=bounds, options={'xtol': 1e-10, 'maxiter': 2000})

# Perform global optimization using basinhopping
# Basin Hopping Methods: Powell
result = sp.optimize.basinhopping(objective_function,x0=x0,minimizer_kwargs={"method": "Powell", "bounds": bounds},niter=1000,T=0.5,stepsize=1,disp=True)

basinhopping step 0: f 0.0108519
basinhopping step 1: f 0.177421 trial_f 0.177421 accepted 1  lowest_f 0.0108519
basinhopping step 2: f 0.00139271 trial_f 0.00139271 accepted 1  lowest_f 0.00139271
found new global minimum on step 2 with function value 0.00139271
basinhopping step 3: f 0.00835037 trial_f 0.00835037 accepted 1  lowest_f 0.00139271
basinhopping step 4: f 6.52151e-15 trial_f 6.52151e-15 accepted 1  lowest_f 6.52151e-15
found new global minimum on step 4 with function value 6.52151e-15
basinhopping step 5: f 0.00838412 trial_f 0.00838412 accepted 1  lowest_f 6.52151e-15
basinhopping step 6: f 0.00692688 trial_f 0.00692688 accepted 1  lowest_f 6.52151e-15
basinhopping step 7: f 0.000128066 trial_f 0.000128066 accepted 1  lowest_f 6.52151e-15
basinhopping step 8: f 0.0832254 trial_f 0.0832254 accepted 1  lowest_f 6.52151e-15
basinhopping step 9: f 0.00867702 trial_f 0.00867702 accepted 1  lowest_f 6.52151e-15
basinhopping step 10: f 0.00551919 trial_f 0.00551919 accepted 1  

C:\Users\Krai\AppData\Local\Temp\ipykernel_40892\96171341.py:23: RuntimeWarning: invalid value encountered in sqrt
  F1 = (N1 * np.sqrt(M1/rho1) / (P1/(rho1 * g))**(3/4)) - (N2 * np.sqrt(M2/rho2) / (P2/(rho2 * g))**(3/4))


basinhopping step 42: f 0.000180309 trial_f 0.000180309 accepted 1  lowest_f 6.52151e-15
basinhopping step 43: f 0.00108742 trial_f 0.00108742 accepted 1  lowest_f 6.52151e-15
basinhopping step 44: f 0.00048032 trial_f 0.00048032 accepted 1  lowest_f 6.52151e-15
basinhopping step 45: f 0.00452448 trial_f 0.00452448 accepted 1  lowest_f 6.52151e-15
basinhopping step 46: f 0.17742 trial_f 0.17742 accepted 1  lowest_f 6.52151e-15
basinhopping step 47: f 0.00139276 trial_f 0.00139276 accepted 1  lowest_f 6.52151e-15
basinhopping step 48: f 0.17742 trial_f 0.17742 accepted 1  lowest_f 6.52151e-15
basinhopping step 49: f 0.0074218 trial_f 0.0074218 accepted 1  lowest_f 6.52151e-15
adaptive stepsize: acceptance rate 0.980000 target 0.500000 new stepsize 1.11111 old stepsize 1
basinhopping step 50: f 0.000560648 trial_f 0.000560648 accepted 1  lowest_f 6.52151e-15
basinhopping step 51: f 0.17742 trial_f 0.17742 accepted 1  lowest_f 6.52151e-15
basinhopping step 52: f 0.00822274 trial_f 0.00822

In [34]:
D2, N2, M2, P2 = result.x[0], result.x[1], result.x[2], result.x[3]
Q2 = M2/rho2
specific_speed_1 = N1*np.sqrt(M1/rho1)/(P1/(rho1*g))**(3/4)
specific_speed_2 = N2*np.sqrt(M2/rho2)/(P2/(rho2*g))**(3/4)
print(f"Ns = {specific_speed_2:.4f}, Matching = {100-abs(specific_speed_1-specific_speed_2)/specific_speed_1*100} %")

flow_coeff_1 = Q1/N1/D1**3
flow_coeff_2 = Q2/N2/D2**3
print(f"Flow Coefficient Prototype = {flow_coeff_1}, Model = {flow_coeff_2}, Matching = {100-abs(flow_coeff_1-flow_coeff_2)/flow_coeff_1*100} %")

head_coeff_1 = g*P1/(rho1*g)/N1**2/D1**2
head_coeff_2 = g*P2/(rho2*g)/N2**2/D2**2
print(f"Head Coefficient Prototype = {head_coeff_1}, Model = {head_coeff_2}, Matching = {100-abs(head_coeff_1-head_coeff_2)/head_coeff_1*100} %")

Reysnold_1 = rho1*N1*D1**2/mu1
Reysnold_2 = rho2*N2*D2**2/mu2
print(f"Reynolds Number Prototype = {Reysnold_1:.4f}, Model = {Reysnold_2:.4f}, Matching = {100-abs(Reysnold_1-Reysnold_2)/Reysnold_1*100} %")

power_coeff_1 = P1 * M1 / rho1 / ( rho1 * N1**3 * D1**5)
power_coeff_2 = P2 * M2 / rho2 / ( rho2 * N2**3 * D2**5)
print(f"Power Coefficient Prototype = {power_coeff_1:.8f}, Model = {power_coeff_2:.8f}, Matching = {100-abs(power_coeff_1-power_coeff_2)/power_coeff_1*100} %")

pump_eff = 0.50
motor_eff = 0.85
mechanical_eff = 0.99

print(f"\nPrototype's Shaft Speed is {N1:.3f} RPM\n"
      f"Prototype's Impeller Diameter is {D1*1e3:.3f} mm\n"
      f"Prototype's Mass Flow Rate is {M1:.3f} Kg/s\n"
      f"Prototype's Pressure Across Pump is {P1/1e5:.3f} Bar\n"
      f"Prototype's Pressure Head is {P1/rho1/9.807:.3f} m HAN+H2O2\n"
      f"Prototype's Pressure Head is {P1/rho1/9.807:.3f} m Testing Fluid\n"
      f"Prototype's Pressure Head is {P1/1000/9.807:.3f} m Water\n"
      f"Prototype's Volumetric Flow Rate is {Q1*1000:.4f} L/s\n"
      f"Prototype's Fluid's Dynamic Viscosity is {mu1*1000:.4f} mPa-s\n"
      f"Prototype's Fluid's Kinematic Viscosity is {mu1/rho1*10**6:.4f} cSt\n"
      f"Prototype's Fluid's Density is {rho1:.4f} kg/m^3\n"
      f"Prototype's Hydraulic Power = {Q1*P1:.3f} W\n"
      f"Prototype's Shaft Power = {Q1*P1/pump_eff/mechanical_eff:.3f} W\n"
      f"Prototype's Shaft Torque = {Q1*P1/pump_eff/mechanical_eff/(N1*2*math.pi/60):.3f} N-m\n"
      f"Prototype's Electrical Power = {Q1*P1/pump_eff/mechanical_eff/motor_eff:.3f} W\n")

print(f"\nModel's Shaft Speed is {N2:.3f} RPM\n"
      f"Model's Impeller Diameter is {D2*1e3:.3f} mm\n"
      f"Model's Mass Flow Rate is {M2:.3f} Kg/s\n"
      f"Model's Pressure Across Pump is {P2/1e5:.3f} Bar\n"
      f"Model's Pressure Head is {P2/rho1/9.807:.3f} m HAN+H2O2\n"
      f"Model's Pressure Head is {P2/rho2/9.807:.3f} m Testing Fluid\n"
      f"Model's Pressure Head is {P2/1000/9.807:.3f} m Water\n"
      f"Model's Volumetric Flow Rate is {Q2*1000:.4f} L/s\n"
      f"Model's Fluid's Dynamic Viscosity is {mu2*1000:.4f} mPa-s\n"
      f"Model's Fluid's Kinematic Viscosity is {mu2/rho2*10**6:.4f} cSt\n"
      f"Model's Fluid's Density is {rho2:.4f} kg/m^3\n"
      f"Model's Hydraulic Power = {Q2*P2:.3f} W\n"
      f"Model's Shaft Power = {Q2*P2/pump_eff/mechanical_eff:.3f} W\n"
      f"Model's Shaft Torque = {Q2*P2/pump_eff/mechanical_eff/(N2*2*math.pi/60):.3f} N-m\n"
      f"Model's Electrical Power = {Q2*P2/pump_eff/mechanical_eff/motor_eff:.3f} W\n")

Ns = 13.7895, Matching = 99.9999999988304 %
Flow Coefficient Prototype = 0.00031263792849786664, Model = 0.0003126379288154082, Matching = 99.99999989843153 %
Head Coefficient Prototype = 0.001366148931251182, Model = 0.0013661489321975372, Matching = 99.99999993072825 %
Reynolds Number Prototype = 836553.8462, Model = 836553.8453, Matching = 99.99999989814819 %
Power Coefficient Prototype = 0.00000043, Model = 0.00000043, Matching = 99.99999982915982 %

Prototype's Shaft Speed is 14000.000 RPM
Prototype's Impeller Diameter is 50.000 mm
Prototype's Mass Flow Rate is 0.850 Kg/s
Prototype's Pressure Across Pump is 10.400 Bar
Prototype's Pressure Head is 68.259 m HAN+H2O2
Prototype's Pressure Head is 68.259 m Testing Fluid
Prototype's Pressure Head is 106.047 m Water
Prototype's Volumetric Flow Rate is 0.5471 L/s
Prototype's Fluid's Dynamic Viscosity is 65.0000 mPa-s
Prototype's Fluid's Kinematic Viscosity is 41.8383 cSt
Prototype's Fluid's Density is 1553.6000 kg/m^3
Prototype's Hydrauli